In [1]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings


from transformers import AutoTokenizer
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast

import pprint
pp = pprint.PrettyPrinter(indent=2)

loader = TextLoader('frontier.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

## Using FORGE-S embeddings

In [2]:
from sentence_transformers import SentenceTransformer, models,util
from transformers import AutoTokenizer
model_path = "/proj/f7b/forge-s-instruct-base1"
word_embedding_model = models.Transformer(model_path, max_seq_length=512)
word_embedding_model.tokenizer.pad_token=word_embedding_model.tokenizer.eos_token
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


In [8]:
word_embedding_model.get_word_embedding_dimension()

2064

In [18]:
e1=model.encode("I am a happy person")
e2=model.encode("the sky is falling")
e3=model.encode("I am a sad person")
e4=model.encode("I am a happy person")
print(util.cos_sim(e1, e2))
print(util.cos_sim(e1, e3))
print(util.cos_sim(e1, e4))

tensor([[0.5146]])
tensor([[0.8933]])
tensor([[1.]])


In [6]:
query_embedding = model.encode('what is the cpu type on frontier')
texts_embedding = model.encode([
    """scheduling policy of frontier is that in a simple batch queue system, jobs run in a first-in, first-out (FIFO) order.""",
    """Frontier is a HPE Cray EX supercomputer located at the Oak Ridge Leadership Computing Facility. """,
    """Each Frontier compute node consists of [1x] 64-core AMD “Optimized 3rd Gen EPYC” CPU (with 2 hardware threads per physical core) with access to 512 GB of DDR4 memory. Each node also contains [4x] AMD MI250X, each with 2 Graphics Compute Dies (GCDs) for a total of 8 GCDs per node.""",
    """system interconnect of frontier is that the Frontier nodes are connected with [4x] HPE Slingshot 200 Gbps (25 GB/s) NICs providing a node-injection bandwidth of 800 Gbps (100 GB/s).""",
    """File systems of frontier is that Frontier is connected to Orion, a parallel filesystem based on Lustre and HPE ClusterStor, with a 679 PB usable namespace (/lustre/orion/).""",
])

print("Similarity:", util.dot_score(query_embedding, texts_embedding))
print("Similarity:", util.cos_sim(query_embedding, texts_embedding))

Similarity: tensor([[42.8058, 51.4530, 49.7797, 31.9822, 40.4576]])
Similarity: tensor([[0.5587, 0.6040, 0.5475, 0.5262, 0.5752]])


## UAE-Large-V1


In [13]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader


In [18]:
from angle_emb import AnglE

angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()
query_embedding = angle.encode('what is the cpu type on frontier')
texts_embedding = angle.encode([
    """scheduling policy of frontier is that in a simple batch queue system, jobs run in a first-in, first-out (FIFO) order.""",
    """Frontier is a HPE Cray EX supercomputer located at the Oak Ridge Leadership Computing Facility. """,
    """Each Frontier compute node consists of [1x] 64-core AMD “Optimized 3rd Gen EPYC” CPU (with 2 hardware threads per physical core) with access to 512 GB of DDR4 memory. Each node also contains [4x] AMD MI250X, each with 2 Graphics Compute Dies (GCDs) for a total of 8 GCDs per node.""",
    """system interconnect of frontier is that the Frontier nodes are connected with [4x] HPE Slingshot 200 Gbps (25 GB/s) NICs providing a node-injection bandwidth of 800 Gbps (100 GB/s).""",
    """File systems of frontier is that Frontier is connected to Orion, a parallel filesystem based on Lustre and HPE ClusterStor, with a 679 PB usable namespace (/lustre/orion/).""",
])

print("Similarity:", util.dot_score(query_embedding, texts_embedding))
print("Similarity:", util.cos_sim(query_embedding, texts_embedding))

Similarity: tensor([[174.6098, 236.6645, 242.2771, 186.1550, 180.4875]])
Similarity: tensor([[0.5718, 0.7040, 0.7826, 0.5898, 0.5810]])
